In [1]:
import re
import os
import sys
import json
import shutil
import hashlib
import platform
import itertools
import collections
import pkg_resources  # pip install py-rouge
from io import open
from konlpy.tag import Mecab 

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
from zipfile import ZipFile

import boto3
from botocore import UNSIGNED
from botocore.client import Config

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

from transformers import PreTrainedTokenizerFast, EarlyStoppingCallback, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

C:\Users\hist\miniconda3\envs\py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Trainer arguments
lr = 1e-4
stop = 3
epoch = 1000
batch = 2
seed = 42
device = 'cuda'

# init

In [3]:
class AwsS3Downloader(object):
    def __init__(
        self,
        aws_access_key_id=None,
        aws_secret_access_key=None,
    ):
        self.resource = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        ).resource("s3")
        self.client = boto3.client(
            "s3",
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            config=Config(signature_version=UNSIGNED),
        )

    def __split_url(self, url: str):
        if url.startswith("s3://"):
            url = url.replace("s3://", "")
        bucket, key = url.split("/", maxsplit=1)
        return bucket, key

    def download(self, url: str, local_dir: str):
        bucket, key = self.__split_url(url)
        filename = os.path.basename(key)
        file_path = os.path.join(local_dir, filename)

        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        meta_data = self.client.head_object(Bucket=bucket, Key=key)
        total_length = int(meta_data.get("ContentLength", 0))

        downloaded = 0

        def progress(chunk):
            nonlocal downloaded
            downloaded += chunk
            done = int(50 * downloaded / total_length)
            sys.stdout.write(
                "\r{}[{}{}]".format(file_path, "█" * done, "." * (50 - done))
            )
            sys.stdout.flush()

        try:
            with open(file_path, "wb") as f:
                self.client.download_fileobj(bucket, key, f, Callback=progress)
            sys.stdout.write("\n")
            sys.stdout.flush()
        except:
            raise Exception(f"downloading file is failed. {url}")
        return file_path

def download(url, chksum=None, cachedir=".cache"):
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    os.makedirs(cachedir_full, exist_ok=True)
    filename = os.path.basename(url)
    file_path = os.path.join(cachedir_full, filename)
    if os.path.isfile(file_path):
        if hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10] == chksum:
            print(f"using cached model. {file_path}")
            return file_path, True

    s3 = AwsS3Downloader()
    file_path = s3.download(url, cachedir_full)
    if chksum:
        assert (
            chksum == hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10]
        ), "corrupted file!"
    return file_path, False

def get_kobart_tokenizer(cachedir=".cache"):
    """Get KoGPT2 Tokenizer file path after downloading"""
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBART/tokenizers/kobart_base_tokenizer_cased_cf74400bce.zip",
        "chksum": "cf74400bce",
    }
    file_path, is_cached = download(
        tokenizer["url"], tokenizer["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    if (
        not os.path.exists(os.path.join(cachedir_full, "emji_tokenizer"))
        or not is_cached
    ):
        if not is_cached:
            shutil.rmtree(
                os.path.join(cachedir_full, "emji_tokenizer"), ignore_errors=True
            )
        zipf = ZipFile(os.path.expanduser(file_path))
        zipf.extractall(path=cachedir_full)
    tok_path = os.path.join(cachedir_full, "emji_tokenizer/model.json")
    tokenizer_obj = PreTrainedTokenizerFast(
        tokenizer_file=tok_path,
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        mask_token="<mask>",
    )
    return tokenizer_obj

def get_pytorch_kobart_model(ctx="cpu", cachedir=".cache"):
    pytorch_kobart = {
        "url": "s3://skt-lsl-nlp-model/KoBART/models/kobart_base_cased_ff4bda5738.zip",
        "chksum": "ff4bda5738",
    }
    model_zip, is_cached = download(
        pytorch_kobart["url"], pytorch_kobart["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    model_path = os.path.join(cachedir_full, "kobart_from_pretrained")
    if not os.path.exists(model_path) or not is_cached:
        if not is_cached:
            shutil.rmtree(model_path, ignore_errors=True)
        zipf = ZipFile(os.path.expanduser(model_zip))
        zipf.extractall(path=cachedir_full)
    return model_path

In [4]:
def make_df(phase):
    work_dir = "C:\\Users\\hist\\Documents\\GitHub\\KoBART"
    if phase == 'train':
        tmp = work_dir+'/022.요약문 및 레포트 생성 데이터/01.데이터/1.Training/라벨링데이터/TL1'
    else:
        tmp = work_dir+'/022.요약문 및 레포트 생성 데이터/01.데이터/2.Validation/라벨링데이터/VL1'
    listdir = os.listdir(tmp)
    df = pd.DataFrame({}, columns = ['genre', 'text', 'label'])
    for i in listdir:
        files = os.listdir(f'{tmp}/{i}/2~3sent')
        for f in tqdm(files):
            with open(f'{tmp}/{i}/2~3sent/{f}', 'r', encoding='utf-8') as json_file:
                j = json.loads(json_file.read())
                df2 = pd.DataFrame.from_dict([{'genre' : i, 
                                               'text'  : j['Meta(Refine)']['passage'], 
                                               'label' : j['Annotation']['summary1']}])
                df = pd.concat([df, df2])
    return df

In [5]:
# %%time

# train = make_df('train').reset_index(drop=True)
# val = make_df('val').reset_index(drop=True)

In [6]:
# train.to_parquet('train.parquet')
# val.to_parquet('val.parquet')

In [7]:
train = pd.read_parquet('train.parquet')
val = pd.read_parquet('val.parquet')
val = val.sample(n=10, replace=False).reset_index(drop=True)
val

,genre,text,label
0,02.briefing,"공정위, 코로나19 피해 사업자의 자료 제출 부담 완화 조치\n 피심인 의견서 제출...",사업자에게 각종 자료 제출의무를 부과하고 있는 공정위는 원칙적으로 기한 내에 자료를...
1,05.minute,"중앙선거관리위원회사무총장 김용희] ""예 맞습니다.""\n임수경 위원] ""이 공문서를 ...",중앙선거관리위원회사무총장 김 씨는 당에서 유권해석을 질의한 것을 사무처의 생각만으로...
2,08.speech,안녕하십니까? 중소기업청 기술협력과장 장대교입니다. 국내 중소기업은 87%가 기술인...,중소기업청은 2012년 중소기업 기술전문가 연계·과제해결 지원 사업을 공고하여 사업...
3,02.briefing,농촌진흥청(청장 김경규)은 봄철 이후 증가하는 변비 증상을 완화하는 약용작물로 참...,농촌진흥청은 봄에 몸속 진액이 말라 배변이 힘들어지는 변비 증상을 약화시키는 약용작...
4,03.his_cul,조선 숙종 때의 문신이자 학자인 노봉 민정중(1628∼1692)의 시문집을 새긴 목...,노봉 민정중의 시문집 목판은 정조 8년 판을 새겼으며 전남 장흥 연곡서원에 소장되어...
5,10.narration,맞은 편에 있는 같은 양식에 있는 건물은 라이트너 박물관이다.\n그 앞에는 1565...,라이트너 박물관 앞에는 스페인 장군 아빌레스의 동상이 서 있다.
6,02.briefing,행정안전부(장관 진영)는 최근 지자체별로 건설이 증가하고 있으나 설계와 유지관리 ...,행정안전부는 지자체별로 건설이 증가하고 있으나 설계와 유지관리 기준이 없어 안전 사...
7,09.literature,"이때에 조정에서는 북벌──북쪽으로 청국을 칠 계획을 세우고, 여러가 지로 준비을 ...",우리를 정복한 것이 청나라나 명나라의 차이가 아니라 항복하여 종노릇 한 것이 부끄러...
8,02.briefing,▶행복도시 산울리(63생활권) 공동주택 설계공모 당선작 발표\n 행정중심복합도시건설...,행복도시 산울리 공동주택 설계공모 당선작은 바람길 확보로 미세먼지 저감을 유도하고 ...
9,05.minute,"법무부장관직무대리 이창재] ""지금 중대한 이익이 있었는지 여부를 포함해서 또 이게 ...",이 법무부장관직무대리는 중대한 이익이 있었는지 소송 대상이 되는지에 대해 법원의 판...


# 전처리

In [8]:
def preprocss(df):
    df.text = df.text.apply(lambda x : re.sub('\n', ' ',  x))
    df.text = df.text.apply(lambda x : re.sub(' +', ' ',  x).strip())
    return df

train = preprocss(train)
val = preprocss(val)

In [9]:
val.loc[0, 'text']

'공정위, 코로나19 피해 사업자의 자료 제출 부담 완화 조치 피심인 의견서 제출 기한도 코로나19 진정 시까지 한시적으로 연장 운영 공정위는 공정한 거래 질서 확립 및 소비자 보호 등을 위하여 소관 법령을 통해 사업자에게 각종 자료 제출 의무를 부과하고 있으며, 법 위반 행위 심의의결 과정에서는, 당사자의 방어권 보장을 위하여 사업자(피심인)에게 해당 사건의 심사 보고서를 송부하면서 이에 대한 의견을 제출할 수 있도록 하고 있다. 최근 코로나19 확산에 따른 사업자들의 정상적인 경제경영 활동에 어려움으로 인하여 일부 사업자가 법에서 정한 기한 내에 자료를 제출하는데 지장을 겪고 있을 것으로 예상되므로, 원칙적으로는 사업자가 기한 내에 자료를 제출하지 않으면 과태료를 부과해야 하나, 최근 우리 기업들이 처해있는 불가피한 사정을 충분히 고려하여 사업자의 부담을 덜어 줄 필요가 있다. 또한, 기업의 재택 근무 확산 등으로 인하여 피심인이 의견 제출을 위한 자료 수집 등에 많은 어려움이 있을 것으로 예상되는 만큼, 기업의 충분한 방어권 보장을 위한 선제적인 조치가 필요하다. 공정위는 코로나19 확산 등에 대응하여, 사업자의 부담 경감을 위한 조치들을 다음과 같이 시행한다. 할부거래법에 따른 감사 보고서 제출(법 제18조의2 및 제53조) (관련 규정) 선불식 할부거래업자(이하 상조업체)는 회계연도 종료 후 3개월 내에 회계 감사 보고서를 공정위에 제출해야 하며, 이를 위반하는 상조업체에는 3,000만 원 이하의 과태료를 부과한다.'

In [10]:
val.loc[0, 'label']

'사업자에게 각종 자료 제출의무를 부과하고 있는 공정위는 원칙적으로 기한 내에 자료를 제출하지 않으면 과태료를 부과하나 코로나19 확산에 대응하여 부담 경감 조치들을 시행한다.'

In [11]:
train.text.str.len().min()

242

# Dataset

In [12]:
class KoBARTSumDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.df = df
        self.len = len(self.df)

        self.pad_index = self.tokenizer.pad_token_id
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs
    
    def __getitem__(self, idx):
        instance = self.df.iloc[idx]
        input_ids = self.tokenizer.encode(instance['text'])
        input_ids = self.add_padding_data(input_ids)

        label_ids = self.tokenizer.encode(instance['label'])
        label_ids.append(self.tokenizer.eos_token_id)
        dec_input_ids = [self.tokenizer.eos_token_id]
        dec_input_ids += label_ids[:-1]
        dec_input_ids = self.add_padding_data(dec_input_ids)
        label_ids = self.add_ignored_data(label_ids)
    
        return {'input_ids': np.array(input_ids, dtype=np.int_),
                'labels': np.array(dec_input_ids, dtype=np.int_),}

    def __len__(self):
        return self.len

In [13]:
train_dataset = KoBARTSumDataset(train, get_kobart_tokenizer(), 256)
val_dataset = KoBARTSumDataset(val, get_kobart_tokenizer(), 256)

using cached model. C:\Users\hist\Documents\GitHub\KoBART\.cache\kobart_base_tokenizer_cased_cf74400bce.zip
using cached model. C:\Users\hist\Documents\GitHub\KoBART\.cache\kobart_base_tokenizer_cased_cf74400bce.zip


In [14]:
train_dataset[0]

{'input_ids': array([16735, 12335, 11821, 14031, 10952, 11841, 11810, 14299, 14143,
        16121, 15991, 19858,  1543, 18044, 11914, 14085, 10770,  9092,
        18025, 10496, 15116, 19211, 14141, 16749, 14862, 14245, 14243,
        16344, 14671, 14725, 14483, 11471,   245, 16735, 10952, 11810,
        17301, 12007, 14067,  9034, 14143, 16121, 15991, 19858, 18463,
        22075, 23658, 14189, 16367, 19775, 11214, 14807, 15425, 25437,
        12005, 21308, 11786, 14114, 12332, 21245, 14174, 11264, 11950,
        14101, 15460, 12074, 14105, 12005, 18817, 11846, 10226, 14130,
        16414, 28403, 19790, 17454, 13469, 17242, 14311, 23449, 25891,
         9714,   243, 14040, 16267, 14075, 10500, 10788, 12060, 13590,
        16338, 11786, 15859, 12007, 16904, 16728, 16261, 22075, 19727,
        16922, 17125, 14130, 26294, 14038, 27368, 14328, 14048, 14038,
        13672, 12333, 14410, 20122, 14736, 18154, 14144, 16626, 14737,
        16527, 15735,  9103, 14159, 14532, 11950, 14130, 14281, 

In [15]:
train.text.str.len().max(), val.text.str.len().max(), 

(1499, 852)

# Model

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained('gogamza/kobart-base-v1').to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [17]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=tokenizer.pad_token_id)

In [20]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    print(tokenizer.decode(labels[0]), tokenizer.decode(preds[0]))

    return {
        "avg cos" : None
    }

In [21]:
args = Seq2SeqTrainingArguments(run_name = f'KoBARTSum',                    # 모델이름
                                output_dir= f"models",                      # 모델저장경로
                                evaluation_strategy="steps",                # 모델의 평가를 언제 진행할지
                                eval_steps=2,                              # 500 스텝 마다 모델 평가
                                save_steps=2,                              # 500 스텝 마다 모델 저장
                                save_total_limit = 2,                       # 저장할 모델의 갯수
                                logging_steps=2,                           # 학습로스 로깅
                                per_device_train_batch_size=batch,          
                                per_device_eval_batch_size=batch,                      
                                gradient_accumulation_steps=16,             # 가상배치
                                num_train_epochs=epoch,                                
                                load_best_model_at_end=True,                # 평가기준 스코어가 좋은 모델만 저장할지 여부
                                fp16=True,
                                do_train=True,
                                do_eval=True,
                                predict_with_generate=True,)

trainer = Seq2SeqTrainer(model=model,
                         tokenizer=tokenizer,
                         args=args,
                         train_dataset=train_dataset,
                         eval_dataset=val_dataset,
                         compute_metrics=compute_metrics,
                         callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],
                         data_collator=collator,) # callback
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
C:\Users\hist\miniconda3\envs\py38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 73340
  Num Epochs = 1000
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 16
  Total optimization steps = 2291000
  Number of trainable parameters = 123859968


Step,Training Loss,Validation Loss,Avg cos
2,2.972100,2.448869,None
4,2.542800,2.123150,None
6,2.167700,1.831461,None
8,1.898400,1.572929,None
10,1.672600,1.333841,None
12,1.392200,1.124837,None
14,1.180700,0.946002,None
16,0.979600,0.796727,None
18,0.823700,0.672356,None
20,0.693900,0.575397,None


***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id"

</s> 사업자에게 각종 자료 제출의무를 부과하고 있는 공정위는 원칙적으로 기한 내에 자료를 제출하지 않으면 과태료를 부과하나 코로나19 확산에 대응하여 부담 경감 조치들을 시행한다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Saving model checkpoint to models\checkpoint-2
Configuration saved in models\checkpoint-2\config.json
Configuration saved in models\checkpoint-2\generation_config.json
Model weights saved in models\checkpoint-2\pytorch_model.bin
tokenizer config file saved in models\checkpoint-2\tokenizer_config.json
Special tokens file saved in models\checkpoint-2\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-20] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  

...

</s> 사업자에게 각종 자료 제출의무를 부과하고 있는 공정위는 원칙적으로 기한 내에 자료를 제출하지 않으면 과태료를 부과하나 코로나19 확산에 대응하여 부담 경감 조치들을 시행한다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Saving model checkpoint to models\checkpoint-18
Configuration saved in models\checkpoint-18\config.json
Configuration saved in models\checkpoint-18\generation_config.json
Model weights saved in models\checkpoint-18\pytorch_model.bin
tokenizer config file saved in models\checkpoint-18\tokenizer_config.json
Special tokens file saved in models\checkpoint-18\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-14] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 10
  Batch size = 2
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id":

</s> 사업자에게 각종 자료 제출의무를 부과하고 있는 공정위는 원칙적으로 기한 내에 자료를 제출하지 않으면 과태료를 부과하나 코로나19 확산에 대응하여 부담 경감 조치들을 시행한다.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Saving model checkpoint to models\checkpoint-20
Configuration saved in models\checkpoint-20\config.json
Configuration saved in models\checkpoint-20\generation_config.json
Model weights saved in models\checkpoint-20\pytorch_model.bin
tokenizer config file saved in models\checkpoint-20\tokenizer_config.json
Special tokens file saved in models\checkpoint-20\special_tokens_map.json


KeyboardInterrupt: 